In [32]:
import pandas as pd
import glob, os
import re
import numpy as np
import orjson 
import pprint
import lzma
import requests

In [130]:
#path = 'enter path of images'
image_files = glob.glob(path+'*.jpg')
image_file_names = [s.strip(path) for s in image_files]

In [131]:
def string_to_int(text):
    return int(text) if text.isdigit() else text
def redefined_keys(text):
    return [string_to_int(c) for c in re.split(r'(\d+)', text)]                                              

In [132]:
image_file_names.sort(key=redefined_keys)
uniqlousa_df = pd.DataFrame(image_file_names, columns=['file_name'])
uniqlousa_df.index = np.arange(1,len(initial_df)+1)
uniqlousa_df['file_name'] = uniqlousa_df['file_name'].map(lambda x: x.rstrip('.jpg'))

In [133]:
file_name = os.listdir(path)
text_files = {}
for file in file_name:
    if file.endswith(".txt"):
        with open(path + file, "r") as target_file:
            text_files[file] = target_file.read()
text_df = (pd.DataFrame.from_dict(text_files, orient='index')
             .reset_index().rename(index = str, columns = {'index': 'file_name', 0: 'caption'}))
text_df['file_name'] = text_df['file_name'].map(lambda x: x.rstrip('.txt'))

In [134]:
uniqlousa_df_consolidated = pd.merge(uniqlousa_df, text_df, on="file_name", how="left")

In [135]:
uniqlousa_df_consolidated

file_name  \
0     2017-06-21_06-19-49_UTC_profile_pic   
1                 2019-03-09_16-23-41_UTC   
2                 2019-03-09_20-23-28_UTC   
3                 2019-03-10_15-52-51_UTC   
4                 2019-03-11_14-00-18_UTC   
...                                   ...   
2258            2021-06-04_13-18-30_UTC_4   
2259            2021-06-04_13-18-30_UTC_5   
2260            2021-06-04_13-18-30_UTC_6   
2261              2021-06-04_19-15-28_UTC   
2262              2021-06-05_14-01-48_UTC   

                                                caption  
0                                                   NaN  
1     Where 5th Ave meets pop art. Now’s your chance...  
2     Wherever you travel, go in style! 🗺️ @elisegio...  
3     Keep it classic. 💯 @brendanfallis in our recen...  
4     Kanpai! Cheers! 🍶 Let's toast to the latest dr...  
...                                                 ...  
2258                                                NaN  
2259                                                NaN  
2260                                                NaN  
2261  Rolling into the weekend like 🤗🚴‍♀️💙 #AIRism #...  
2262  Want to keep your weekend relaxed? We've got t...  

[2263 rows x 2 columns]

In [45]:
path = ''
file_src = os.listdir(path)

In [36]:
#Turn the json parsing into a script for automated csv file generation with all the datapoints
columns = ['type', 'shortcode', 'file_name', 'comments']
df = pd.DataFrame(columns=[])
#interesting datapoints : accessibility_caption, shortcode,  locations_id, locations_name, 
#mentions, hastags, edge_media_to_tagged_user, edge_media_preview_like, 

In [91]:
for file in file_src:  
    if file.endswith('C.json.xz'):       
        with open(path + file, "r") as target_file:
            metadata_file = target_file.name
            metadata_file_names = metadata_file.strip(path)
            unpack_metadata = lzma.open(metadata_file, mode='rt', encoding='utf-8').read()
            metadata = orjson.loads(unpack_metadata)
            post_type = metadata['node']['__typename']
            shortcode = metadata['node']['shortcode']
            #captions = metadata['node']['edge_media_to_caption']
            metadata_dict = {'filename':metadata_file_names, 'format-of-post':post_type, 'shortcode':shortcode}
            df = df.append(metadata_dict, ignore_index=True)
            del metadata
            target_file.close()

In [99]:
df.to_csv("sample_consolidated_metadata.csv", index=False)

In [100]:
check_sample = pd.read_csv("sample_consolidated_metadata.csv")
check_sample

filename format-of-post    shortcode
0   2020-10-21_17-48-16_UTC.json.xz     GraphVideo  CGnU9D-Jvcz
1   2021-01-12_15-21-40_UTC.json.xz     GraphImage  CJ8yQ5cMfqo
2   2021-03-24_13-27-56_UTC.json.xz   GraphSidecar  CMzZr_Xs5OK
3   2020-08-01_14-05-13_UTC.json.xz     GraphImage  CDWXJicFq5q
4   2020-08-16_20-18-11_UTC.json.xz     GraphImage  CD9pwSEFIJQ
..                              ...            ...          ...
65  2020-02-18_23-25-44_UTC.json.xz     GraphVideo  B8ugGfCFdt8
66  2020-11-22_21-33-23_UTC.json.xz     GraphVideo  CH6IPojFFnG
67  2020-07-17_23-15-04_UTC.json.xz     GraphVideo  CCwsQuSlxw0
68  2021-04-03_18-37-21_UTC.json.xz     GraphVideo  CNNtAu4DmnF
69  2020-09-25_23-18-49_UTC.json.xz     GraphVideo  CFk-NOMFLaf

[70 rows x 3 columns]